In [4]:
import pandas as pd
pd.set_option("display.max_rows", 200)
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import datetime
import os

import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# data load and merge

## filedir set

In [2]:
# 唯一需要更改的地方
# dirname 指输出的目录
dirname = "./S_R8/"
st_dir = "/public/hanl/jupyter_dir/ir_recommend_v2/02.state/S_merge.csv"
ac_dir = "/public/hanl/jupyter_dir/ir_recommend_v2/01.action/"
r_dir = "/public/hanl/jupyter_dir/ir_recommend_v2/03.reward/R_v8.csv"

## fea

In [39]:
df_fea = pd.read_csv(st_dir)
df_fea = df_fea.sort_values(by=["stay_id", "starttime"])
#df_fea["starttime"] = pd.to_datetime(df_fea["starttime"], format='%Y/%m/%d %H:%M:%S')
#df_fea["endtime"] = pd.to_datetime(df_fea["endtime"], format='%Y/%m/%d %H:%M:%S')

print(df_fea.shape)
df_drop = df_fea[df_fea.duplicated(["stay_id", "starttime", "endtime"])]
df_fea = pd.concat([df_fea, df_drop, df_drop]).drop_duplicates(keep="first") 
print(df_fea.shape)

df_trajid = pd.read_csv(ac_dir + "A_IR_win60.csv")
df_trajid = df_trajid.loc[:,["stay_id", "starttime", "endtime", "traj_id"]]
df_fea = pd.merge(df_fea, df_trajid, how="left", on=["stay_id", "starttime", "endtime"])
print(df_fea.shape)

tmp_df = pd.DataFrame()
for i,j in df_fea.groupby("traj_id"):
    j = j.sort_values("starttime")
    j["next_glucose_max"] = j["glucose_max"].shift(-1)
    j["next_glucose_min"] = j["glucose_min"].shift(-1)
    j["next_glucose_mean"] = j["glucose_mean"].shift(-1)
    j = j.fillna(method="ffill")
    tmp_df = pd.concat([tmp_df, j])
df_fea = tmp_df
print(df_fea.shape)
df_fea = df_fea[~df_fea["next_glucose_min"].isna()] # 取出traj_id 只有一个状态的轨迹
df_fea.to_csv("tmp_df_fea.csv",index=False)


(261756, 264)
(261747, 264)
(261750, 265)
(261750, 268)


In [37]:
# stat
tmp_df = pd.DataFrame(df_fea.isna().sum())
tmp_df.columns = ["count"]
tmp_df.sort_values("count")

,count
stay_id,0
L_v8_1,0
L_v9_0,0
L_v9_1,0
L_v10_0,0
...,...
OS_v7_2,0
OS_v8_0,0
OS_v8_1,0
OS_v3_2,0


## reward

In [ ]:
### add last rate
df_trajid = pd.read_csv(ac_dir + "A_IR_win60.csv")
df_trajid = df_trajid.loc[:,["stay_id", "starttime", "endtime", "traj_id","rate"]]

tmp_df = pd.DataFrame()
for i,j in df_trajid.groupby("traj_id"):
    j = j.sort_values("starttime")
    j["last_rate"] = j["rate"].shift(+1)
    j = j.fillna(method="bfill")
    tmp_df = pd.concat([tmp_df, j])
df_trajid = tmp_df
#df_trajid.to_csv("tmp_A_IR_win60_addlast_rate.csv",index=False)

# click here

In [139]:
df_trajid = pd.read_csv("tmp_A_IR_win60_addlast_rate.csv")

df_ac = pd.read_csv(r_dir)
df_ac = pd.merge(df_ac, df_trajid, how="left", on=["stay_id", "starttime", "endtime", "rate","traj_id"] )
df_ac = df_ac.sort_values(["traj_id", "starttime"])
df_ac.head(1)

,reward,stay_id,starttime,endtime,rate,traj_id,last_rate
0,5.3125,30001148,2156-08-30 20:15:00,2156-08-30 21:15:00,2.0,1,2.0


## merge

In [143]:
df_fea = pd.read_csv("tmp_df_fea.csv")
print(df_fea.shape)
# merge
df_ac_fea = pd.merge( df_fea, df_ac, how="left", on=["stay_id", "starttime", "endtime", "traj_id"])
df_ac_fea["starttime"] = pd.to_datetime(df_ac_fea["starttime"], format='%Y/%m/%d %H:%M:%S')
df_ac_fea["endtime"] = pd.to_datetime(df_ac_fea["endtime"], format='%Y/%m/%d %H:%M:%S')
df_ac_fea = df_ac_fea.drop_duplicates()
df_ac_fea = df_ac_fea.sort_values(["traj_id", "starttime"])
print(df_ac_fea.shape)

(261745, 268)
(261749, 271)


# 将时间转为入ICU后的第几分钟

In [144]:
datadir = "/public/hanl/jupyter_dir/ir_recommend_v2/00.data/"
file = datadir + "icustays_pat_with_r_interval.csv"
df_icu = pd.read_csv(file)
df_icu["intime"] = pd.to_datetime(df_icu["intime"], format='%d/%m/%Y %H:%M:%S' )
df_icu=df_icu.loc[:,["stay_id", "intime"]]

In [145]:
tmp_df = pd.merge(df_ac_fea, df_icu, how="left",on="stay_id")
tmp_df["starttime_m"] = (tmp_df["starttime"] - tmp_df["intime"]).map(lambda x:x/np.timedelta64(1, 'm')).tolist()
tmp_df["endtime_m"] = (tmp_df["endtime"] - tmp_df["intime"]).map(lambda x:x/np.timedelta64(1, 'm')).tolist()
tmp_df = tmp_df.drop(["starttime","endtime","intime"], axis=1)
tmp_df = tmp_df.rename(columns={'starttime_m': 'starttime', "endtime_m":'endtime'})

df_ac_fea = tmp_df
df_ac_fea["starttime"] = round(df_ac_fea["starttime"],2)
df_ac_fea["endtime"] = round(df_ac_fea["endtime"],2)

In [146]:
df_ac_fea["idx"] = range(len(df_ac_fea))
df_ac_fea_raw = df_ac_fea
print(df_ac_fea_raw.shape)
df_ac_fea = df_ac_fea[~df_ac_fea["reward"].isna()]
print(df_ac_fea.shape)

(261749, 272)
(207586, 272)


# action bin

In [147]:
cut_point = [0, 0.25,1.75, 2.75, 3.75, 4.75, 5.75, 6.75, 7.75, 8.75, 9.75, 10.75, 11.75, 12.75, 13.75, 16]

max_rate = max(df_ac_fea["rate"])

rate_bin = {}
tmp_rate = cut_point[0]
cut_point = cut_point[1:]

idx = 0
while tmp_rate <= max_rate:
    last_tmp_rate = tmp_rate
    tmp_rate = cut_point[idx]
    rate_bin[(last_tmp_rate, tmp_rate)] = idx
    idx += 1
print(rate_bin)

rate_bin2 ={}
lst = df_ac_fea["rate"].unique().tolist()
for i in lst:
    for keys in rate_bin.keys():
        if (i >= keys[0]) & (i<keys[1]):
            rate_bin2[i] = rate_bin[keys]

{(0, 0.25): 0, (0.25, 1.75): 1, (1.75, 2.75): 2, (2.75, 3.75): 3, (3.75, 4.75): 4, (4.75, 5.75): 5, (5.75, 6.75): 6, (6.75, 7.75): 7, (7.75, 8.75): 8, (8.75, 9.75): 9, (9.75, 10.75): 10, (10.75, 11.75): 11, (11.75, 12.75): 12, (12.75, 13.75): 13, (13.75, 16): 14}


In [148]:
df_ac_fea["rate_bin"] = df_ac_fea.rate.map(rate_bin2)
df_ac_fea.index = pd.RangeIndex(len(df_ac_fea.index))

In [137]:
tmp_df = pd.DataFrame(columns=["rate_bin", "insulin_rate_range", "count", "percent"])
rate_bin_reverse = {}
for i,j in rate_bin.items():
    rate_bin_reverse[j] = i
for i,j in df_ac_fea.groupby("rate_bin"):
    tmp_df.loc[len(tmp_df)] = [i,rate_bin_reverse[i], j.shape[0], round(j.shape[0]/df_ac_fea.shape[0],2)]
tmp_df.sort_values(by=["count"],ascending=False)

,rate_bin,insulin_rate_range,count,percent
2,2,"(1.75, 2.75)",47252,0.23
3,3,"(2.75, 3.75)",36239,0.17
1,1,"(0.25, 1.75)",34246,0.16
4,4,"(3.75, 4.75)",28064,0.14
5,5,"(4.75, 5.75)",20629,0.10
6,6,"(5.75, 6.75)",13419,0.06
7,7,"(6.75, 7.75)",8184,0.04
8,8,"(7.75, 8.75)",7011,0.03
10,10,"(9.75, 10.75)",4081,0.02
9,9,"(8.75, 9.75)",3468,0.02


# 划分数据集

##  Split data by action

In [150]:
config = {'random_SEED': 44,  
          'train_sample': 0.8,  # percentage (0 to 1) of patients reserved for training (remainder used for valiation)
          'val_sample':0.2,
          'test_sample': 0,
         }
config_df = pd.DataFrame(config, index=[0])

In [151]:
df_ac_fea = df_ac_fea.sort_values(["traj_id","starttime"])
tmp_df = df_ac_fea

import random
random.seed(config['random_SEED'])

####### MIMIC data SPLIT
# now split into train/validation/test sets
unique_ids = tmp_df['traj_id'].unique()
random.shuffle(unique_ids)
train_sample = config['train_sample']
val_sample = config['val_sample']

train_num = int(len(unique_ids) * train_sample)
val_num = int(len(unique_ids) * val_sample) + train_num
test_num = int(len(unique_ids) * 0.1 ) #test_sample)

train_ids = unique_ids[:train_num]
val_ids = unique_ids[train_num:val_num]
test_ids = unique_ids[val_num:]

# Create datasets
train_set = tmp_df.loc[tmp_df['traj_id'].isin(train_ids)]
val_set = tmp_df.loc[tmp_df['traj_id'].isin(val_ids)]
test_set = tmp_df.loc[tmp_df['traj_id'].isin(test_ids)]

print(train_set.shape)
print(val_set.shape)
print(test_set.shape)

train_set.index = pd.RangeIndex(len(train_set.index))
val_set.index = pd.RangeIndex(len(val_set.index))
test_set.index = pd.RangeIndex(len(test_set.index))

(165794, 273)
(41780, 273)
(12, 273)


## Save to Pickle and csv

In [152]:
data_dir = "/public/hanl/jupyter_dir/ir_recommend_v2/04.merge/"

if not os.path.exists(os.path.join(data_dir, dirname)):
    os.makedirs(os.path.join(data_dir, dirname))
    
data_dir2 = data_dir + "/" +dirname + "/"

# Save processed data to csv files
df_ac_fea_raw.to_csv(os.path.join(data_dir2, 'all_data.csv'), index=False)

train_set.to_csv(os.path.join(data_dir2, 'train_data.csv'), index=False)
val_set.to_csv(os.path.join(data_dir2, 'val_data.csv'), index=False)
test_set.to_csv(os.path.join(data_dir2, 'test_data.csv'), index=False)

# df_ac_fea_raw 中包含了所有的状态，便于后面状态转移，
# train_set，val_set，test_set中只包含了有Rreward的状态，有些SAR因为真实glucose的缺乏导致R没有